In [ ]:
# import libraries
import BioSimSpace as BSS
import os
import sys
import glob
import csv
import numpy as np
import networkx as nx
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale, MinMaxScaler
import itertools as it

import pipeline

from pipeline.prep import *
from pipeline.utils import *

pipeline.__file__

import random
import math
import pandas as pd
import subprocess

fwf_path = (
    "/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis"
)
if fwf_path not in sys.path:
    sys.path.insert(1, fwf_path)

import networkanalysis

from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

from scipy.stats import sem as sem
from scipy.stats import bootstrap, norm
from scipy.stats import spearmanr
import scipy.stats as _stats

from sklearn.preprocessing import minmax_scale

In [ ]:
protein = "mcl1"

exec_folder = f"/home/anna/Documents/benchmark/reruns/{protein}/execution_model"

##### <span style="color:teal">Comparing lomap and the rbfenn</span>  


In [ ]:
# get a list of the perts in each and all together
perts_lomap = []
perts_rbfenn = []
perts = []

file_a = f"{exec_folder}/network_lomap.dat"
# file_a = f"a-optimal-lomap.csv"
file_b = f"{exec_folder}/network_rbfenn.dat"
# file_b = f"d-optimal-lomap.csv"

perts_rbfenn, ligs = pipeline.utils.get_info_network(file_a)
perts_lomap, ligs = pipeline.utils.get_info_network(file_b)
perts = perts_rbfenn + perts_lomap

# write a file that contains the combined perts, directions are distinct
combined_perts = []
filtered_out = 0
filtered_perts = []
for pert in perts:
    if not pert in combined_perts:
        combined_perts.append(pert)
    else:
        filtered_out += 1
        filtered_perts.append(pert)
print(
    f"Removed {filtered_out} duplicate perts between lomap and rbfenn to give {len(combined_perts)} combined perts."
)

# write a file that contains the unique perts, 1 direction only.
filtered_perts = []
filtered_out = 0
for pert in combined_perts:
    inv_pert = pert.split("~")[1] + "~" + pert.split("~")[0]

    if not pert in filtered_perts and not inv_pert in filtered_perts:
        filtered_perts.append(pert)
    else:
        filtered_out += 1
print(
    f"Removed {filtered_out} inverse perts to give {len(filtered_perts)} unique perts, one direction only."
)

# get the perts that are unique to each
unique_perts = []
unique_out_lomap = 0
unique_out_rbfenn = 0
shared_out = 0

for pert in perts_lomap:
    inv_pert = pert.split("~")[1] + "~" + pert.split("~")[0]

    if not pert in perts_rbfenn and not inv_pert in perts_rbfenn:
        unique_perts.append((pert, "lomap"))
        unique_out_lomap += 1

for pert in perts_rbfenn:
    inv_pert = pert.split("~")[1] + "~" + pert.split("~")[0]

    if not pert in perts_lomap and not inv_pert in perts_lomap:
        unique_perts.append((pert, "rbfenn"))
        unique_out_rbfenn += 1

for pert in combined_perts:
    inv_pert = pert.split("~")[1] + "~" + pert.split("~")[0]

    if pert in perts_lomap or inv_pert in perts_lomap:
        if pert in perts_rbfenn or inv_pert in perts_rbfenn:
            unique_perts.append((pert, "shared"))
            shared_out += 1


print(
    f"There are {unique_out_lomap} pert(s) unique to lomap and {unique_out_rbfenn} pert(s) unique to rbfenn."
)
print(f"There are {shared_out} pert(s) shared between lomap and rbfenn")

In [ ]:
from pipeline.utils._network import get_info_network

prot_dict = {}
for prot in ["tyk2", "mcl1", "p38"]:  # , "cmet", "syk", "hif2a"
    prot_dict[prot] = {}
    for network in ["lomap", "rbfenn", "flare"]:
        try:
            exec_folder = (
                f"/home/anna/Documents/benchmark/reruns/{prot}/execution_model"
            )
            perts, ligs = get_info_network(
                net_file=f"{exec_folder}/network_{network}.dat",
            )
            prot_dict[prot][network] = len(perts) / len(ligs)
        except:
            prot_dict[prot][network] = 0
# for prot in ["cmet", "syk", "hif2a"]:
#     prot_dict[prot]["flare"] = 0

In [ ]:
df = pd.DataFrame.from_dict(prot_dict).T
df.plot.bar(
    color=["darkslateblue", "purple", "orchid", "lavender"],
)
plt.legend(loc="lower right")
plt.title("perturbations/ligands")

In [ ]:
# manually combined all perts,
perts_all, ligs = pipeline.utils.get_info_network("tyk2_all_perts.dat")
print(len(perts_all))
# print(perts_all)

all_perts = []

for pert in perts_all:
    inv_pert = pert.split("~")[1] + "~" + pert.split("~")[0]

    if pert in combined_perts or inv_pert in combined_perts:
        all_perts.append((pert, "old"))
    else:
        all_perts.append((pert, "new"))

df = pd.DataFrame(all_perts)

df.loc[df[1] == "new"]

In [ ]:
with open(f"{exec_folder}/combined_perts.dat", "w") as writefile:
    writer = csv.writer(writefile)
    for pert in combined_perts:
        writer.writerow([pert])
print(f"Total number of combined perturbations: {len(combined_perts)}")

# with open(f"{exec_folder}/filtered_perts.dat", "w") as writefile:
#     writer = csv.writer(writefile)
#     for pert in filtered_perts:
#         writer.writerow([pert])
# print(f"Total number of filtered perturbations: {len(filtered_perts)}")

# # write a file for the different perts
# with open(f"{exec_folder}/unique_perts.dat", "w") as writefile:
#     writer = csv.writer(writefile)
#     for pert in unique_perts:
#         writer.writerow([pert[0], pert[1]])
# print(
#     f"Total number of unique perturbations: {len(unique_perts)} (lomap: {unique_out_lomap}, rbfenn: {unique_out_rbfenn})"
# )
# print(f"Total number of shared perturbations: {shared_out}")

### generate a and d optimal networks

scripts are in R, adapted from Yang et al. and run seperately.

In [ ]:
protein = "p38"

exec_folder = f"/home/anna/Documents/benchmark/reruns/{protein}/execution_model"
# exec_folder = f"/home/anna/Documents/benchmark/{protein}_benchmark/execution_model"

In [ ]:
def scaleArray(arr):
    """Scales an array to be the inverse in the range [0-1]."""

    # normalise to the range 0-1.
    return minmax_scale(1 / arr, feature_range=(0.03, 1))

In [ ]:
# to generate a network, need to make matrix
# RBFENN links file needs to be generated using the RBFENN_setup.ipynb

links_file = f"{exec_folder}/RBFENN/links_file.in"

# rbfenn file

value_dict = {}

# need to normalise the rbfenn so less close to 0 as otherwise is not able to det(matrix) way more than 0 when using the R script.

with open(links_file, "r") as f:
    for line in f:
        lig0 = line.split(" ")[0].strip()
        lig1 = line.split(" ")[1].strip()
        score = line.split(" ")[2].strip()
        if lig0 not in value_dict.keys():
            value_dict[lig0] = {}
            value_dict[lig0][lig1] = score
        else:
            value_dict[lig0][lig1] = score

pd.options.display.float_format = "{:,.5f}".format
df = pd.DataFrame(value_dict).fillna(1)

df = df.sort_index()
df = df.reindex(sorted(df.columns), axis=1)

# df.to_csv(
#     f"{exec_folder}/{protein}_rbfenn_matrix.csv",
#     index_label="ID",
# )
df.to_csv(
    f"/home/anna/Documents/other_workflows/yang2020_optimal_designs/me/{protein}/{protein}_rbfenn_matrix.csv",
    index_label="ID",
)

In [ ]:
# lomap file

value_dict = {}

# initialise pipeline at top of file
pl = initialise_pipeline()
# where the ligands for the pipeline are located. These should all be in the same folder in sdf format
pl.ligands_folder(f"/home/anna/Documents/benchmark/inputs/reruns/{protein}/ligands")
# where the pipeline should be made
pl.main_folder("/home/anna/Documents/benchmark/test")
pl.setup_ligands()

for lig0, lig1 in it.product(
    list(pl.ligands_dict.keys()), list(pl.ligands_dict.keys())
):
    if lig0 != lig1:
        print(lig0, lig1)
        single_transformation, score = BSS.Align.generateNetwork(
            [pl.ligands_dict[lig0], pl.ligands_dict[lig1]],
            names=[lig0, lig1],
            plot_network=False,
            links_file=None,
        )
        if lig0 not in value_dict.keys():
            value_dict[lig0] = {}
            value_dict[lig0][lig1] = score[0]
        else:
            value_dict[lig0][lig1] = score[0]

df = pd.DataFrame(value_dict).fillna(1)

In [ ]:
# arr = scaleArray(df.to_numpy())
# df_arr = pd.DataFrame(arr, columns=list(pl.ligands_dict.keys()), index=list(pl.ligands_dict.keys()))
df_arr = df

# df.to_csv(f"{exec_folder}/{protein}_lomap_matrix.csv", index_label="ID")
df_arr.to_csv(
    f"/home/anna/Documents/other_workflows/yang2020_optimal_designs/me/{protein}/{protein}_lomap_matrix.csv",
    index_label="ID",
)